In [ ]:
import pandas as pd

In [ ]:
# 加载样本

train_data = pd.read_csv('./sample/train_data_sample.csv')
test_data = pd.read_csv('./sample/test_data_sample.csv')
user_info = pd.read_csv('./sample/user_info_sample.csv')
user_log = pd.read_csv('./sample/user_log_sample.csv')


In [ ]:
# 数据处理
user_log.rename(columns={'seller_id': 'merchant_id'}, inplace=True)
user_log['brand_id'].fillna(0, inplace=True)
user_log['brand_id'] = user_log['brand_id'].astype('int64')
user_log['time_stamp'] = pd.to_datetime(user_log['time_stamp'], format='%H%M')
user_info['age_range'].fillna(0, inplace=True)
user_info = user_info.replace([8], [7])
user_info['age_range'] = user_info['age_range'].astype('int64')
user_info['gender'].fillna(2, inplace=True)
user_info['gender'] = user_info['gender'].astype('int64')

In [ ]:
train_data['origin'] = 'train'
test_data['origin'] = 'test'
matrix = pd.concat([train_data, test_data], ignore_index=True, sort=False)
matrix.drop('prob', axis=1, inplace=True)
matrix = matrix.merge(user_info, on='user_id', how='left')

In [ ]:
# 用户特征处理
groups = user_log.groupby(['user_id'])
temp = groups.size().reset_index().rename(columns={0: 'u1'})
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['item_id'].agg([('u2', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['cat_id'].agg([('u3', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['merchant_id'].agg([('u4', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['brand_id'].agg([('u5', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['time_stamp'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['u6'] = (temp['L_time'] - temp['F_time']).dt.seconds/3600
matrix = matrix.merge(temp[['user_id', 'u6']], on='user_id', how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0: 'u7', 1: 'u8', 2: 'u9', 3: 'u10'})
matrix = matrix.merge(temp, on='user_id', how='left')

# 商户特征处理
groups = user_log.groupby(['merchant_id'])
temp = groups.size().reset_index().rename(columns={0: 'm1'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = groups['user_id', 'item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'user_id': 'm2', 'item_id': 'm3', 'cat_id': 'm4', 'brand_id': 'm5'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0: 'm6', 1: 'm7', 2: 'm8', 3: 'm9'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = train_data[train_data['label'] == -1].groupby('merchant_id').size().reset_index().rename(columns={0: 'm10'})
matrix = matrix.merge(temp, on='merchant_id', how='left')

In [ ]:
groups = user_log.groupby(['user_id', 'merchant_id'])
temp = groups.size().reset_index().rename(columns={0: 'um1'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id': 'um2', 'cat_id': 'um3', 'brand_id': 'um4'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0: 'um5', 1: 'um6', 2: 'um7', 3: 'um8'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['time_stamp'].agg([('first', 'min'), ('last', 'max')]).reset_index()
temp['um9'] = (temp['last'] - temp['first']).dt.seconds/3600
temp.drop(['last', 'first'], axis=1, inplace=True)
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')

In [ ]:
matrix['r1'] = matrix['u9'] / matrix['u7']
matrix['r2'] = matrix['m8'] / matrix['m6']
matrix['r3'] = matrix['um7'] / matrix['um5']
matrix.fillna(0, inplace=True)
temp = pd.get_dummies(matrix['age_range'], prefix='age')
matrix = pd.concat([matrix, temp], axis=1)
temp = pd.get_dummies(matrix['gender'], prefix='gen')
matrix = pd.concat([matrix, temp], axis=1)
matrix.drop(['age_range', 'gender'], axis=1, inplace=True)